# 프로그램 컨셉
이 프로그램은 코스피200선물 거래를 위해서 tensorflow 홈페이지에 있는 A2C (advantage actor-critic) sample을 기반으로 작성되었다.

이미 DNN으로 학습된 앙상블 모델의 포지션에서 actor-critic에 의해 산텍된 거래수량 증감에 따른 수익률의 변화를 순차적으로 n개를 나열한 것과 현재 포지션을 state로 하여 actor와 critic의 공통 입력 데이터로 시용한다.

actor의 output인 action은 거래 수량. 주어진 모델의 기대 수익이 critic의 목표값이 된다.

# Actor-Critic 메서드로 코스피200선물 거래하기 - DeepMoney

**Actor-Critic 방법**

Actor-Critic 방법은 가치 함수와 독립적인 정책 함수를 나타내는 [Temporal Difference(TD) 학습](https://en.wikipedia.org/wiki/Temporal_difference_learning) 방법입니다.

정책 함수(또는 정책)는 에이전트가 주어진 상태에 따라 취할 수 있는 동작에 대한 확률 분포를 반환합니다. 가치 함수는 주어진 상태에서 시작하여 특정 정책에 따라 영원히 동작하는 에이전트의 예상 이익을 결정합니다.

Actor-Critic 방법에서 정책은 주어진 상태에 따라 가능한 일련의 동작을 제안하는 *행위자*라고 하며, 추정값 함수는 주어진 정책에 따라 *행위자*가 취한 동작을 평가하는 *비평가*라고 합니다.

이 튜토리얼에서 *행위자*와 *비평가* 모두 두 개의 출력이 있는 하나의 신경망을 사용하여 표현됩니다.


딥러닝으로 이미 학습된 모델을 이용하여 시그널을 발생시킨다. 그 시그널에 의해 거래하는데 action은 거래량 또는 손절 비율이 된다.
코스피 200 선물 - 60분봉 데이터 - 딥러닝의 시그널과 actor의 거래량에 의해 거래한 결과 (예)

* 0 : 중립,  1: 매도,   2: 매수

date      거래 시그널	종가	수익	수수료        거래량
2022/04/19/13:00	1	357.5	0	0               1
2022/04/19/14:00	1	357.25	0	0               0
2022/04/19/15:00	2	357.2	75000	5360.25     2  
2022/04/20/09:00	0	355.7	0	0               0
2022/04/20/10:00	2	355.45	0	0               0
2022/04/20/11:00	0	356	0	0                   0     
2022/04/20/12:00	0	357.1	0	0
2022/04/20/13:00	0	357.7	0	0
2022/04/20/14:00	0	357.45	500000	5346.75
2022/04/20/15:00	2	356.95	0	0
2022/04/21/09:00	0	359.85	0	0
2022/04/21/10:00	2	360.4	0	0
2022/04/21/11:00	2	360.2	0	0
2022/04/21/12:00	0	359.45	0	0
2022/04/21/13:00	0	359.7	0	0
2022/04/21/14:00	0	359.65	-187500	5400.375
2022/04/21/15:00	2	358.9	0	0
2022/04/22/09:00	1	353.75	0	0              2 
2022/04/22/10:00	1	354.6	0	0              0


## 설정

필요한 패키지를 가져오고 전역 설정을 구성합니다.


In [1]:
import collections
import numpy as np
import pandas as pd
import statistics
import tensorflow as tf
import tqdm
import pandas as pd
import os

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

# 60분봉 데이터 load
df0 = pd.read_csv("kospi200f_60M.csv", encoding='euc-kr')

# 앙상블 모델의 거래 데이터 로드
profit_df = pd.read_csv("ensemble_profits_2021~2024-02-23.csv", encoding='euc-kr')

In [2]:
ensembles = [
["25HL", "30C", "40P", 0.02, 7, 28],
]

In [3]:
import data
from data import config
conf = config()
import ensemble_proc2 as ep

import pandas as pd
import datetime

conf.input_size = 83
conf.df0_path = 'kospi200f_60M.csv'  # 원본 파일
conf.df_pred_path = 'kospi200f_60M_pred.csv'  # 예측용 normalization file
conf.result_path = 'pred_83_results.csv'  # 예측 결과 손익 파일

def ensemble_predict(start_time, end_time, ensemble):
    
    start_rate = profit_df.loc[profit_df['dates'] >= start_time].min()[ensemble]
    end_rate = profit_df.loc[profit_df['dates'] <= end_time].max()[ensemble]
    
    return end_rate - start_rate

def ensemble_signal(conf, end_time, last_train, ensemble, tradiing_num, loss_cut):
    
    conf.end_time = end_time
    conf.start_time = (datetime.datetime.strptime(conf.end_time, '%Y/%m/%d/%H:%M') - datetime.timedelta(days=15)).strftime("%Y/%m/%d/09:00")
    conf.last_train = last_train    

    conf.selected_model_types = ensemble[:3]
    
    conf.reinfo_th = ensemble[3]
    conf.pred_term = ensemble[4]
    conf.reinfo_width = ensemble[5]
    
    conf.gubun = 0
    conf.result_path = 'pred_83_results.csv'
    
    conf.trading_num = trading_num
    conf.loss_cut = loss_cut

    data.preprocessing(conf)
    r = ep.predict(conf)

    signal = pd.read_csv(conf.result_path).values[-1, 1]
    
    return signal



'\ndef ensemble_signal(conf, end_time, last_train, ensemble):\n    \n    conf.end_time = end_time\n    conf.start_time = (datetime.datetime.strptime(conf.end_time, \'%Y/%m/%d/%H:%M\') - datetime.timedelta(days=15)).strftime("%Y/%m/%d/09:00")\n    conf.last_train = last_train    \n\n    conf.selected_model_types = ensemble[:3]\n    \n    conf.reinfo_th = ensemble[3]\n    conf.pred_term = ensemble[4]\n    conf.reinfo_width = ensemble[5]\n    \n    conf.gubun = 0\n    conf.result_path = \'pred_83_results.csv\'\n\n    data.preprocessing(conf)\n    r = ep.predict(conf)\n\n    signal = pd.read_csv(conf.result_path).values[-1, 1]\n    \n    return signal\n'

In [4]:
data.preprocessing(conf)

## 코스피200 선물 파일 address

원본 파일, 정규화 파일(학습용, 예측 용), 예측 결과 파일

conf.df0_path = 'kospi200f_60M2.csv'  # 원본 파일
conf.df_pred_path = 'kospi200f_60M_pred.csv'  # 예측용 normalization file
conf.result_path = 'pred_83_results.csv'  # 예측 결과 손익 파일

## 모델

*행위자*와 *비평가*는 각각 동작 확률과 비평 값을 생성하는 하나의 신경망을 사용하여 모델링됩니다. 이 튜토리얼에서는 모델 하위 클래스화를 사용하여 모델을 정의합니다.

순방향 전달 중에 모델은 상태를 입력으로 받고 상태 종속 값을 모델링하는 동작 확률과 비평 값 $V$를 모두 출력합니다. 목표는 예상 이익을 최대화하는 $\pi$ 정책을 기반으로 행동을 선택하는 모델을 훈련하는 것입니다.

deepmoney의 경우, 상태를 나타내는 4 가지 값이 있는데, 각각 20일 수익률(%), 5일 수익률(%), 1일 수익률(%) 및 보유 상태(0: 없음, 1:매도, 2: 매수)입니다. 에이전트는 중립(0, 거래없음), 매도(1, 청산후 진입)와 매수(2, 청산후 진입) 3가지 동작을 취할 수 있습니다.

In [5]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common1 = layers.Dense(num_hidden_units, activation="relu")
    self.common2 = layers.Dense(int(num_hidden_units/2), activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x1 = self.common1(inputs)
    x2 = self.common2(x1)
    return self.actor(x2), self.critic(x2)

In [6]:
num_actions = 3  # 2
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

# 거래 환경 제공 및 훈련 데이터 생성

2022-01-01~ 2023-01-20 까지의 코스피200 선물 가격 데이터로부터 모델의 action(매수/매도 거래 시그널)과 현재 state에 의한 수익을 평가하고 5일, 20일 평균 수익을 계산하여 새로운 state를 반환한다.

In [7]:
import datetime

class env:
  """Combined actor-critic network."""

  def __init__(self, conf): 

    """Initialize."""
    self.conf = conf
    self.start_time = conf.start_time
    self.end_time = conf.end_time
    self.last_train = conf.last_train
    self.pred_term = conf.pred_term
    self.initial_pos = 0
    self.current_pos = int(0)
    self.df = pd.read_csv(conf.df0_path, encoding='euc-kr')
    self.price = self.df['종가'].values
    self.state = tf.constant(np.zeros(23), dtype=tf.float32)
    
  def step(self, action: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:

    if self.current_pos > len(self.df) - self.pred_term:
        return self.state, tf.constant(0)
    
    # new state 초기화
    new_state  = np.zeros(23)
    new_profit = np.zeros(20)
       
    # 현재 position 이동
    self.current_pos += self.pred_term
    self.current_pos = min(self.current_pos, len(self.df) - 1)
    
    # 단기, 중기, 장기 수익률 비율 state 생성
    for i in range(20):
        self.end_time = self.df['date'].values[self.current_pos]
        self.end_time = self.end_time[:11] + "15" + self.end_time[13:]
        self.start_time = (datetime.datetime.strptime(self.end_time, '%Y/%m/%d/%H:00') - datetime.timedelta(days=5) ).strftime('%Y/%m/%d/09:00')
        self.start_time = self.df.loc[self.df['date'] <= self.start_time].max().values[2]
        self.start_time = self.start_time[:11] + "09" + self.start_time[13:]
        self.start_time = max(self.df['date'].values[0], self.start_time)

        current_profit = ensemble_predict(self.start_time, self.end_time, "model" + str(i))

        self.end_time = (datetime.datetime.strptime(self.end_time, '%Y/%m/%d/%H:00') - datetime.timedelta(days=1) ).strftime('%Y/%m/%d/09:00')
        self.end_time = self.df.loc[self.df['date'] <= self.end_time].max().values[2]
        self.end_time = self.start_time[:11] + "15" + self.end_time[13:]
        self.end_time = max(self.df['date'].values[0], self.end_time)
        
        self.start_time = (datetime.datetime.strptime(self.end_time, '%Y/%m/%d/%H:00') - datetime.timedelta(days=5) ).strftime('%Y/%m/%d/09:00')
        self.start_time = self.df.loc[self.df['date'] <= self.start_time].max().values[2]
        self.start_time = self.start_time[:11] + "09" + self.start_time[13:]
        self.start_time = max(self.df['date'].values[0], self.start_time)

        pre_profit = ensemble_predict(self.start_time, self.end_time, "model" + str(i))
        
  
        # 5일 수익률의 일 변화 
        new_state[i] = current_profit - pre_profit
        
    # 단기, 중기, 장기 수익률 state에 첨가
    new_state[20] = (self.price[self.current_pos] - self.price[max(0, self.current_pos-35)]) / self.price[max(0, self.current_pos-35)]
    new_state[21] = (self.price[self.current_pos] - self.price[max(0, self.current_pos-140)]) / self.price[max(0, self.current_pos-140)]
    new_state[22] = (self.price[self.current_pos] - self.price[max(0, self.current_pos-420)]) / self.price[max(0, self.current_pos-420)]
    
    # reward는 포지션 이동 전후의 수익률 변화 벡분율
    end_time = self.df['date'].values[self.current_pos]
    end_time = self.start_time[:11] + "15" + self.end_time[13:]
    end_time = max(self.df['date'].values[0], self.end_time)

    start_time = self.df['date'].values[self.current_pos - self.pred_term]
    start_time = self.start_time[:11] + "09" + self.start_time[13:]
    start_time = max(self.df['date'].values[0], self.start_time)
    
    reward = (ensemble_predict(start_time, end_time, "model" + str(action)) - 1  ) * 100
    
    # 변경된 state 저장
    self.state = tf.convert_to_tensor(new_state)
    
    return self.state, reward

  def reset(self, conf) -> tf.Tensor:
    self.sate = tf.constant(np.zeros(23), dtype=tf.float32)
    self.conf = conf
    self.start_time = conf.start_time
    self.end_time = conf.end_time
    self.last_train = conf.last_train
    self.pred_term = conf.pred_term
    self.current_pos = self.df.loc[self.df['date'] >= self.start_time].index.min() + 420
    return self.state

## 에이전트 훈련

에이전트를 훈련하기 위해 다음 단계를 따릅니다.

1. 환경에서 에이전트를 실행하여 에피소드별로 훈련 데이터를 수집합니다.
2. 각 시간 스텝에서 예상 이익을 계산합니다.
3. 결합된 Actor-Critic 모델의 손실을 계산합니다.
4. 그래디언트를 계산하고 네트워크 매개변수를 업데이트합니다.
5. 성공 기준 또는 최대 에피소드에 도달할 때까지 1~4를 반복합니다.


### 1. 훈련 데이터 수집

지도 학습에서와 같이 Actor-Critic 모델을 훈련하려면 훈련 데이터가 필요합니다. 그러나, 이러한 데이터를 수집하려면 모델이 환경에서 "실행"되어야 합니다.

여기서는 각 에피소드에 대한 훈련 데이터를 수집합니다. 그런 다음, 모델의 가중치에 의해 매개변수화된 현재 정책을 기반으로 동작 확률과 비평 값을 생성하기 위해 각 타임스텝에서 모델의 순방향 전달을 환경 상태에서 실행합니다.

다음 동작은 모델에 의해 생성된 동작 확률로부터 샘플링되며, 그런 다음 환경에 적용되어 다음 상태와 보상을 생성합니다.

이 프로세스는 더 빠른 훈련을 위해 나중에 TensorFlow 그래프로 컴파일할 수 있도록 TensorFlow 연산을 사용하는 `run_episode` 함수에서 구현됩니다. `tf.TensorArray`는 가변 길이 배열에서 Tensor 반복을 지원하는 데 사용되었습니다.

In [8]:
# initiate env to apply a step of action
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward = env.step(action)
  return (np.array(state, np.float32), 
          np.array(reward, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32])

In [9]:
env = env(conf)

In [10]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
  
    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])
  
    # Apply action to the environment to get next state and reward
    state, reward = tf_env_step(action)
    state.set_shape(initial_state_shape)
  
    # Store reward
    rewards = rewards.write(t, reward)


  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()
  
  return action_probs, values, rewards

### 2. 예상 이익 계산

한 에피소드 동안 수집된 각 타임스텝 $t$, ${r_{t}}^{T}*{t=1}$에서 보상의 시퀀스를 예상 이익 ${G*{t}}^{T}_{t=1}$의 시퀀스로 변환합니다. 여기서 보상의 합계는 현재 타임스텝 $t$에서 $T$까지 계산되며, 각 보상에 기하급수적으로 감소하는 할인 계수 $\gamma$를 곱합니다.

$$G_{t} = \sum^{T}_{t'=t} \gamma^{t'-t}r_{t'}$$

$\gamma\in(0,1)$ 이후, 현재 타임스텝에서 더 멀리 떨어진 보상에는 더 적은 가중치가 부여됩니다.

직관적으로, 예상 이익은 단순히 지금 보상이 이후 보상보다 낫다는 것을 암시합니다. 이것은 수학적 의미에서 보상의 합이 수렴하도록 하려는 것입니다.

To stabilize training, the resulting sequence of returns is also standardized (i.e. to have zero mean and unit standard deviation).


In [11]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

### 3. Actor-Critic 손실

하이브리드 Actor-Critic 모델을 사용하고 있기 때문에, 아래와 같이 훈련을 위해 Actor와 Critic 손실의 조합인 손실 함수를 사용합니다.

$$L = L_{actor} + L_{critic}$$

#### Actor 손실

[비평가가 상태 종속 기준선인 정책 그래디언트](https://www.youtube.com/watch?v=EKqxumCuAAY&t=62m23s)를 기반으로 행위자 손실을 공식화하고 단일 샘플(에피소드별) 추정치를 계산합니다.

$$L_{actor} = -\sum^{T}_{t=1} \log\pi_{\theta}(a_{t} | s_{t})[G(s_{t}, a_{t})  - V^{\pi}_{\theta}(s_{t})]$$

여기서:

- $T$: 에피소드별로 달라질 수 있는 에피소드별 타임스텝의 수
- $s_{t}$: $t$ 타임스텝의 상태
- $a_{t}$: $s$ 상태에 따라 $t$ 타임스텝에서 선택된 동작
- $\pi_{\theta}$: $\theta$에 의해 매개변수화된 정책(Actor)
- $V^{\pi}_{\theta}$: 마찬가지로 $\theta$에 의해 매개변수화된 값 함수(Critic)
- $G = G_{t}$: 주어진 상태에 대한 예상 이익, 타임스텝 $t$에서 동작 쌍

A negative term is added to the sum since the idea is to maximize the probabilities of actions yielding higher rewards by minimizing the combined loss.

<br>

##### 이점

$L_{actor}$ 공식에서 $G - V$ 항을 [이점](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#advantage-functions)이라고 하며, 이는 특정한 상태에서 $\pi$ 정책에 따라 선택된 임의의 동작보다 이 상태에 얼마나 더 나은 동작이 주어지는지를 나타냅니다.

기준선을 제외할 수 있지만 이로 인해 훈련 중에 큰 변동이 발생할 수 있습니다. 그리고 비평가 $V$를 기준선으로 선택할 때의 좋은 점은 가능한 한 $G$에 가깝게 훈련되어 변동이 낮아진다는 것입니다.

또한, Critic이 없으면 알고리즘이 예상 이익을 바탕으로 특정 상태에서 취하는 행동의 확률을 높이려고 시도할 것이며, 이 때 동작 사이의 상대적 확률이 같게 유지된다면 큰 차이가 생기지 않습니다.

예를 들어, 주어진 상태에서 두 행동의 예상 이익이 같다고 가정합니다. Critic이 없으면 알고리즘은 목표 $J$에 따라 이들 동작의 확률을 높이려고 합니다. Critic의 경우, 이점($G - V = 0$)이 없기 때문에 동작의 확률을 높이는 데 따른 이점이 없으며 알고리즘이 그래디언트를 0으로 설정합니다.

<br>

#### The Critic loss

$V$를 $G$에 최대한 가깝게 훈련하는 것은 다음 손실 함수를 사용한 회귀 문제로 설정할 수 있습니다.

$$L_{critic} = L_{\delta}(G, V^{\pi}_{\theta})$$

여기서 $L_{\delta}$는 [Huber 손실](https://en.wikipedia.org/wiki/Huber_loss)로, 제곱 오차 손실보다 데이터의 이상 값에 덜 민감합니다.


In [12]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

### 4. 매개변수를 업데이트하기 위한 훈련 단계 정의

위의 모든 단계를 모든 에피소드에서 실행되는 훈련 단계로 결합합니다. 손실 함수로 이어지는 모든 단계는 `tf.GradientTape` 컨텍스트로 실행되어 자동 미분이 가능합니다.

이 튜토리얼에서는 Adam 옵티마이저를 사용하여 모델 매개변수에 그래디언트를 적용합니다.

할인되지 않은 보상의 합계인 `episode_reward`도 이 단계에서 계산됩니다. 이 값은 나중에 성공 기준이 충족되는지 평가하는 데 사용됩니다.

`tf.function` 컨텍스트를 `train_step` 함수에 적용하여 호출 가능한 TensorFlow 그래프로 컴파일할 수 있고, 그러면 훈련 속도가 10배 빨라질 수 있습니다.


In [13]:

#@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate the expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

### 5. 훈련 루프 실행하기

성공 기준 또는 최대 에피소드 수에 도달할 때까지 훈련 단계를 실행하는 방식으로 훈련을 실행합니다.

대기열을 사용하여 에피소드 보상의 실행 레코드를 유지합니다. 100회 시도에 도달하면 가장 오래된 보상이 대기열의 왼쪽(꼬리쪽) 끝에서 제거되고 최근 보상이 머리쪽(오른쪽)에 추가됩니다. 계산 효율을 높이기 위해 보상의 누적 합계도 유지됩니다.

런타임에 따라 훈련은 1분 이내에 완료될 수 있습니다.

In [14]:
import random

def train(conf, train_df, model):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    env.df = train_df
    env.price = train_df['종가'].values
    
    min_episodes_criterion = 30
    max_episodes = 1000
    max_steps_per_episode = 100 #500

    # `CartPole-v1` is considered solved if average reward is >= 475 over 500 
    # consecutive trials
    reward_threshold = 5
    running_reward = 0

    # The discount factor for future rewards
    gamma = 0.99

    # Keep the last episodes reward
    episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

    t = tqdm.trange(max_episodes)
    for i in t:
        
        conf.start_time = train_df['date'].values[0]
        initial_state = env.reset(conf)
        #initial_state = tf.constant(initial_state, dtype=tf.float64)
        
        env.current_pos = random.randint(420, len(env.price) - max_steps_per_episode - env.pred_term - 1)
        
        episode_reward = int(train_step(
            initial_state, model, optimizer, gamma, max_steps_per_episode))

        episodes_reward.append(episode_reward)
        running_reward = statistics.mean(episodes_reward)


        t.set_postfix(
            episode_reward=episode_reward, running_reward=running_reward)

        # Show the average episode reward every 10 episodes
        if i % 10 == 0:
          pass # print(f'Episode {i}: average reward: {avg_reward}')

        if running_reward > reward_threshold and i >= min_episodes_criterion:  
            break

    print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')
    
    return model

### 6. test

test(test dataframe, trained_model)

 - environment setup
 - make_actions로 시그널 생성
 - 주어진 지수 데이터 범위에서 수익 계산

make_actions: 주어진 가격 데이터 범위 (env.price) 내의 데이터들에 대한 actor의 action, 즉 모델의 선택을 생성한다.

make_signals: 모델 선택에 의해 차례대로 test data에서 시그널을 생성한다.

In [15]:
def make_actions(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    """Runs env step as many as mas steps."""

    actions = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)

    initial_state_shape = initial_state.shape
    state = initial_state

    for t in tf.range(max_steps):
        # 마지막 step의 out of bounds를 막기 위하여
        env.current_pos = min(env.current_pos, max_steps - 2)
        
        # Convert state into a batched tensor (batch size = 1)
        state = tf.expand_dims(state, 0)

        # Run the model and to get action probabilities and critic value
        action_logits_t, value = model(state)

        # Sample next action from the action probability distribution
        action = tf.random.categorical(action_logits_t, 1)[0, 0]

        # Store actions
        actions = actions.write(t, action)

        # Apply action to the environment to get next state and reward
        state, reward = tf_env_step(action)
        state.set_shape(initial_state_shape)

    actions = actions.stack()
  
    return actions

"""
# 모델 선텍에 의한 시그널 생성
def make_signals(conf, actions, max_steps):
    
    initial_state = env.reset(conf)
    
    signals = [] 
    for i in range(max_steps):

        ensemble = ensembles[actions[i]]
        signals.append(ensemble_signal(conf, start_time, last_train, ensemble))
    
    return signals
"""

# 모델 선텍에 의한 시그널 생성
def selected_action_signals(conf, action):
    
    #conf.last_train = "F:/알고리즘트레이딩4-1/" + conf.last_train

    ensemble = ensembles[action]
    signals = ensemble_signals(conf, ensemble)
    
    return signals

def ensemble_signals(conf, ensemble):
    
    conf.selected_model_types = ensemble[:3]
    
    conf.reinfo_th = ensemble[3]
    conf.pred_term = ensemble[4]
    conf.reinfo_width = ensemble[5]
    
    conf.gubun = 0
    conf.result_path = 'pred_88_results.csv'

    data.preprocessing(conf)
    r = ep.predict(conf)

    signals = pd.read_csv(conf.result_path).values[:, 1]
    
    return signals

    

test: 주어진 학습 모델과 기간의 dataframe을 이용하여 해당 기간의 수익을 make_action의 return 시그널에 근거하여 계산해서 반환, 

In [16]:
def test(conf, test_df, model):
    
    env.df = test_df
    env.price = env.df['종가'].values
    
    conf.start_time = test_df['date'].values[0]
    initial_state = env.reset(conf)
    initial_state = tf.constant(initial_state, dtype=tf.float64)

    max_steps = len(env.price)

    actions = make_actions(initial_state, model, max_steps)
    signals = selected_action_signals(conf, actions[0])

    # 15시 종가를 익일 시가로 조정
    #for i in range(len(df)-1):
    #    if i != 0 and df.loc[i, 'date'][11:13] == '15':
    #        df.loc[i, 'close'] = df.loc[i+1, 'open']

    state = 0
    count = 0
    buy_price = 0
    profit = []
    fee = []

    loss_cut = 0.01
    pred_term = 5

    for i in range(len(signals)):

        pred = signals[i]
        close = float(test_df['종가'].values[i])
        high = float(test_df['고가'].values[i])
        low = float(test_df['저가'].values[i])
        open = float(test_df['시가'].values[i])
        date = test_df['date'].values[i]

        if date[11:13] == '15':
            if state == 1:
                profit.append((buy_price - close) * 250000)
                fee.append((buy_price + close) * 250000 * 0.00003)
            elif state == 2:
                profit.append((close - buy_price) * 250000)
                fee.append((buy_price + close) * 250000 * 0.00003)
            else:
                profit.append(0)
                fee.append(0)

            state = 0
            count = 0
            buy_price = 0

            continue

        if state == 1:

            if high - buy_price >= buy_price * loss_cut:
                profit.append(-int((buy_price*loss_cut+0.05)/0.05)*0.05*250000)
                fee.append((buy_price + close)*250000*0.00003)
                state = pred
                count = 1
                buy_price = close
            elif pred == 2:
                profit.append((buy_price - close)*250000)
                fee.append((buy_price + close)*250000*0.00003)
                state = pred
                count = 1
                buy_price = close
            else:
                profit.append(0)
                fee.append(0)
                count += 1
                count %= pred_term
        elif state == 2:

            if buy_price - low >= buy_price * loss_cut:
                profit.append(-int((buy_price*loss_cut+0.05)/0.05)*0.05*250000)
                fee.append((buy_price + close)*250000*0.00003)
                state = pred
                count = 1
                buy_price = close
            elif pred == 1:
                profit.append((close - buy_price)*250000)
                fee.append((buy_price + close)*250000*0.00003)
                state = pred
                count = 1
                buy_price = close
            else:
                profit.append(0)
                fee.append(0)
                count += 1
                count %= pred_term
        else:
            if pred == 1 or pred == 2:
                state = pred
                count = 1
                buy_price = close
            else:
                count = 0
            profit.append(0)
            fee.append(0)
            
    print((sum(profit) - sum(fee))/test_df['종가'].values.mean()/1.25/250000/0.09 + 1)
    
    dic = {'date': test_df['date'].values, 'close': test_df['종가'].values, 'profit': profit, 'fee': fee}

    return pd.DataFrame(dic)

# incrementa learning

전체 구간을 1개월 단위로 이동하면서 train, test하여 각 test 결과들을 합쳐서 전체 test 결과를 보여준다. 

In [17]:
train_start = ['2021/01/01/09:00', '2021/02/01/09:00', '2021/03/01/09:00', '2021/04/01/09:00', '2021/05/01/09:00',
               '2021/06/01/09:00', '2021/07/01/09:00', '2021/08/01/09:00', '2021/09/01/09:00', '2021/10/01/09:00',
               '2021/11/01/09:00', '2021/12/01/09:00']

train_end = ['2021/12/31/15:00', '2022/01/31/15:00', '2022/02/28/15:00', '2022/03/31/15:00', '2022/04/30/15:00',
             '2022/05/31/15:00', '2022/06/30/15:00', '2022/07/31/15:00', '2022/08/31/15:00', '2022/09/30/15:00',
             '2022/10/31/15:00', '2022/11/30/15:00']

test_end = ['2022/01/31/15:00', '2022/02/28/15:00', '2022/03/31/15:00', '2022/04/30/15:00', '2022/05/31/15:00',
            '2022/06/30/15:00', '2022/07/31/15:00', '2022/08/31/15:00', '2022/09/30/15:00', '2022/10/31/15:00',
            '2022/11/30/15:00', '2022/12/31/15:00']
"""
train_start = ['2021/01/01/09:00']
train_end = ['2023/05/31/15:00']
test_start = ['2023/06/01/09:00']
test_end = ['2023/06/30/15:00']
"""
# model 초기화
model = ActorCritic(num_actions, num_hidden_units)

all_return_df = pd.DataFrame(columns=['date', 'close', 'profit', 'fee'])

for i in range(len(train_start)):
    # train dataframe 생성
    train_df = df0.loc[df0['date'] >= train_start[i]].reset_index()
    train_df = train_df.loc[train_df['date'] <= train_end[i]].reset_index()

    # 모델 training
    conf.start_time = train_start
    conf.end_time = train_end
    model = train(conf, train_df, model)

    # trained model save
    model.save('models/' + train_end[i][:10].replace('/', '-'))
    #model.save('models/2023-12-31')

    # test dataframe 생성
    test_df = df0.loc[df0['date'] > train_end[i]].reset_index()
    test_df = test_df.loc[test_df['date'] <= test_end[i]].reset_index()

    # test 후 결과 append
    conf.start_time = test_df['date'].values[0]
    conf.end_time = test_df['date'].values[-1]

    # last_train 설정: test 시작일까지 최근 학습된 날짜로
    if conf.start_time[8:10] > '15':
        conf.last_train = conf.start_time[:4] + '-' + conf.start_time[5:7] + '-' + '15'
    else:
        conf.last_train = (datetime.datetime.strptime(conf.start_time, '%Y/%m/%d/%H:%M') - datetime.timedelta(days=15)).strftime("%Y-%m-%d")
        if conf.last_train[5:7] == '02':
            conf.last_train = conf.last_train[:8] + '28'
        elif conf.last_train[5:7] == '04' or conf.last_train[5:7] == '06' or conf.last_train[5:7] == '09' or conf.last_train[5:7] == '11':
            conf.last_train = conf.last_train[:8] + '30'
        else:
            conf.last_train = conf.last_train[:8] + '31'
    conf.last_train = "F:/알고리즘트레이딩2-1/" + conf.last_train
    
    all_return_df = all_return_df.append(test(conf, test_df, model))

all_return_df.to_csv('test_results_2022-01-01~2022-12-31.csv', index=False)
    
print((all_return_df['profit'].values.sum() - all_return_df['fee'].values.sum())/all_return_df['close'].values.mean()/1.25/250000/0.09 +1)

  6%|██▏                                 | 60/1000 [16:26<4:17:32, 16.44s/it, episode_reward=6679, running_reward=54.2]



Solved at episode 60: average reward: 54.20!
INFO:tensorflow:Assets written to: models/2021-12-31\assets

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD652148> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E288>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E288> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E548>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E548> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD550B88>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD552988> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD642D08>).

Two checkpoint references resolved to different objec

c:\users\user\anaconda3\envs\tensorflow2.4\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\users\user\anaconda3\envs\tensorflow2.4\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221811EED48> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221811EEC08>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221811EEC08> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FF85E288>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FF85E288> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD55D0C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD652148> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E288>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD64E288> and <tensorflow.python.keras.

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

...

  2%|▌                                   | 17/1000 [01:21<1:20:20,  4.90s/it, episode_reward=-519, running_reward=-627]

  4%|█▎                                  | 37/1000 [03:02<1:19:18,  4.94s/it, episode_reward=4149, running_reward=23.9]



Solved at episode 37: average reward: 23.93!
INFO:tensorflow:Assets written to: models/2022-11-30\assets

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218755D288> and <tensorflow.python.keras.layers.core.Dense object at 0x0000022187568748>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x0000022187568748> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221875C8688>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221875C8688> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F6668C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218123F788> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F67C1C8>).

Two checkpoint references resolved to different objec

c:\users\user\anaconda3\envs\tensorflow2.4\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\users\user\anaconda3\envs\tensorflow2.4\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218755D288> and <tensorflow.python.keras.layers.core.Dense object at 0x0000022187568748>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x0000022187568748> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221875C8688>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221875C8688> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F6668C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x0000022185595608> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F677E08>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218F677E08> and <tensorflow.python.keras.

In [18]:
env.end_time

'2022/12/29/15:00'

In [19]:
env.df = test_df
env.price = env.df['종가'].values

conf.start_time = test_df['date'].values[0]
initial_state = env.reset(conf)
initial_state = tf.constant(initial_state, dtype=tf.float64)

max_steps = len(env.price)

actions = make_actions(initial_state, model, max_steps)

conf.end_time = test_df['date'].values[-1]
conf.lend_time = test_df['date'].values[-1]
conf.last_train = "F:/알고리즘트레이딩2-1/" + "2024-01-31"
signals = selected_action_signals(conf, actions[0])
print(signals)

nothing done! in this preprocessing

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD53FC88> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD77B688>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD77B688> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD76D308>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000221FD76D308> and <tensorflow.python.keras.layers.core.Dense object at 0x00000221FD54A4C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218F789688> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F6BCD48>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218F6


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218F661CC8> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F7613C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002218F7613C8> and <tensorflow.python.keras.layers.core.Dense object at 0x0000022185502208>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x0000022185502208> and <tensorflow.python.keras.layers.core.Dense object at 0x000002218F782BC8>).
[0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1
 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 2 1 1 1 0 0 1 1 1 1 2 0 1 1 0 1 1 0 2 0 1 2
 2 0 0 1 1 0 1 1 1 1 0 1 2 0 1 0 0 0 0 0 2 2 1 1 0 1 1 0 1 1 1 1 1 1 0 0]


In [20]:
ensemble_predict('2024/01/01/09:00', '2024/02/07/15:00', 9)

0.4800026967132114

## 다음 단계

이 튜토리얼에서는 Tensorflow를 사용하여 Actor-Critic 방법을 구현하는 방법을 보여주었습니다.

다음 단계로 Gym의 다른 환경에서 모델의 훈련을 시도할 수 있습니다.

Actor-Critic 방법 및 Cartpole-v0 문제에 대한 추가 정보는 다음 리소스를 참조하세요.

- [Actor-Critic 메서드](https://hal.inria.fr/hal-00840470/document)
- [Actor-Critic 강의(CAL)](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=7&t=0s)
- [Cart Pole 학습 제어 문제 [Barto 등 1983]{/a}](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf)

TensorFlow에서 더 많은 강화 학습 예를 보려면 다음 리소스를 확인하세요.

- [강화 학습 코드 예제(keras.io)](https://keras.io/examples/rl/)
- [TF-Agents 강화 학습 라이브러리](https://www.tensorflow.org/agents)
